In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256, 256], [512, 512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [32]:
size = 5

## Random Projections

In [11]:
(
    swmhau_network_grp_kfold_5,
    best_swmhau_network_grp_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_5_kfold_best_model.csv


In [12]:
swmhau_network_grp_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1956874/572217191.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.701460   
                                                                                      0.0005         0.697732   
                                                                                      0.0010         0.691581   
                                                                         0.2          0.0001         0.694812   
                                                                                      0.0005         0.691208   
                                                                                      0.0010         0.694564   
                                                          (512, 512)     0.1          0.0001         0.695247   
                                                                                      0.0005         0.699099   
                                                                                      0.0010         0.695682   
                                                                         0.2          0.0001         0.693880   
                                                                                      0.0005         0.692700   
                                                                                      0.0010         0.694688   
           12              3         10        1          (256, 256)     0.1          0.0001         0.695806   
                                                                                      0.0005         0.696303   
                                                                                      0.0010         0.695993   
                                                                         0.2          0.0001         0.695744   
                                                                                      0.0005         0.693880   
                                                                                      0.0010         0.694439   
                                                          (512, 512)     0.1          0.0001         0.692513   
                                                                                      0.0005         0.693383   
                                                                                      0.0010         0.696490   
                                                                         0.2          0.0001         0.691084   
                                                                                      0.0005         0.694066   
                                                                                      0.0010         0.697359   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.674144   
                                                                                      0.0005         0.675148   
                                                                                      0.0010         0.668930   
                                                                         0.2          0.0001         0.671502   
                                                                                      0.0005         0.669537   
                                                                                      0.0010         0.671402   
                                                          (512, 512)     0.1          0.0001         0.670430   
                                                                                      0.0005         0.674427   
                                

In [13]:
best_swmhau_network_grp_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.694501,0.670427,"[0.759501100513573, 0.5813537675606641]",0.668134,"[0.777877968139465, 0.5583905789990187]",0.674130,"[0.7419724770642202, 0.6062866275972296]",None,0.731365,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.695806,0.672507,"[0.7598587404355504, 0.585155058464667]",0.670042,"[0.7805320435308344, 0.5595527467185221]",0.676732,"[0.7402522935779816, 0.6132125732551944]",None,0.730287,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.698602,0.672523,"[0.7649367640645443, 0.5801090625811477]",0.670867,"[0.7758773223237982, 0.5658561296859169]",0.674700,"[0.7543004587155964, 0.5950985615343634]",None,0.727996,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_grp_kfold_5[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0005
0,15,12,3,10,1,"(256, 256)",0.1,0.0005
0,15,12,3,10,1,"(256, 256)",0.1,0.0005


in path: 0.6670696726194932 (3)

in input: 0.6743158202211089 (1)

both: 0.6718190822700244 (2)

neither: 0.6002572169116608

In [15]:
best_swmhau_network_grp_kfold_5["f1"].mean()

0.6718190822700244

In [16]:
best_swmhau_network_grp_kfold_5["precision"].mean()

0.6696811315662593

In [17]:
best_swmhau_network_grp_kfold_5["recall"].mean()

0.6751871652907643

In [18]:
np.stack(best_swmhau_network_grp_kfold_5["f1_scores"]).mean(axis=0)

array([0.7614322 , 0.58220596])

In [19]:
np.stack(best_swmhau_network_grp_kfold_5["precision_scores"]).mean(axis=0)

array([0.77809578, 0.56126649])

In [20]:
np.stack(best_swmhau_network_grp_kfold_5["recall_scores"]).mean(axis=0)

array([0.74550841, 0.60486592])

# w=11

In [33]:
size = 11

## Random Projections

In [34]:
(
    swmhau_network_grp_kfold_11,
    best_swmhau_network_grp_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_11_kfold_best_model.csv


In [35]:
swmhau_network_grp_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1956874/2867464325.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.683691   
                                                                                      0.0005         0.691084   
                                                                                      0.0010         0.692016   
                                                                         0.2          0.0001         0.681019   
                                                                                      0.0005         0.689717   
                                                                                      0.0010         0.691146   
                                                          (512, 512)     0.1          0.0001         0.683628   
                                                                                      0.0005         0.693942   
                                                                                      0.0010         0.691581   
                                                                         0.2          0.0001         0.679963   
                                                                                      0.0005         0.695371   
                                                                                      0.0010         0.699658   
           12              3         10        1          (256, 256)     0.1          0.0001         0.678907   
                                                                                      0.0005         0.693507   
                                                                                      0.0010         0.696117   
                                                                         0.2          0.0001         0.675116   
                                                                                      0.0005         0.693880   
                                                                                      0.0010         0.690214   
                                                          (512, 512)     0.1          0.0001         0.678409   
                                                                                      0.0005         0.692265   
                                                                                      0.0010         0.698975   
                                                                         0.2          0.0001         0.686921   
                                                                                      0.0005         0.694750   
                                                                                      0.0010         0.699969   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.666689   
                                                                                      0.0005         0.668611   
                                                                                      0.0010         0.666516   
                                                                         0.2          0.0001         0.664768   
                                                                                      0.0005         0.668858   
                                                                                      0.0010         0.667755   
                                                          (512, 512)     0.1          0.0001         0.664070   
                                                                                      0.0005         0.669989   
                                

In [36]:
best_swmhau_network_grp_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.681827,0.663253,"[0.7423396226415094, 0.5841656516443361]",0.661010,"[0.7838699394325789, 0.5381508078994613]",0.671887,"[0.7049885321100917, 0.6387852956846031]",None,0.775441,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.681267,0.659892,"[0.7451564828614008, 0.5746268656716418]",0.657440,"[0.7759155803848541, 0.5389640690620625]",0.666043,"[0.716743119266055, 0.6153436334576452]",None,0.786764,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.673625,0.658249,"[0.730739658619099, 0.585758220960492]",0.657434,"[0.7880597014925373, 0.5268085106382979]",0.670378,"[0.6811926605504587, 0.6595631326584976]",None,0.785820,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [37]:
best_swmhau_network_grp_kfold_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


in path: 0.6614111415863579

in input: 0.6320948773281413

both: 0.6604644170664132

In [38]:
best_swmhau_network_grp_kfold_11["f1"].mean()

0.6604644170664132

In [39]:
best_swmhau_network_grp_kfold_11["precision"].mean()

0.6586281014849654

In [40]:
best_swmhau_network_grp_kfold_11["recall"].mean()

0.6694360622878919

In [41]:
np.stack(best_swmhau_network_grp_kfold_11["f1_scores"]).mean(axis=0)

array([0.73941192, 0.58151691])

In [42]:
np.stack(best_swmhau_network_grp_kfold_11["precision_scores"]).mean(axis=0)

array([0.78261507, 0.53464113])

In [43]:
np.stack(best_swmhau_network_grp_kfold_11["recall_scores"]).mean(axis=0)

array([0.70097477, 0.63789735])

# w=20

In [44]:
size = 20

## Random Projections

In [45]:
(
    swmhau_network_grp_kfold_20,
    best_swmhau_network_grp_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_20_kfold_best_model.csv


In [46]:
swmhau_network_grp_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1956874/1955217834.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_20.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.673563   
                                                                                      0.0005         0.694874   
                                                                                      0.0010         0.692700   
                                                                         0.2          0.0001         0.665797   
                                                                                      0.0005         0.689966   
                                                                                      0.0010         0.688102   
                                                          (512, 512)     0.1          0.0001         0.676546   
                                                                                      0.0005         0.694315   
                                                                                      0.0010         0.699907   
                                                                         0.2          0.0001         0.676918   
                                                                                      0.0005         0.695868   
                                                                                      0.0010         0.694191   
           12              3         10        1          (256, 256)     0.1          0.0001         0.675738   
                                                                                      0.0005         0.677415   
                                                                                      0.0010         0.705250   
                                                                         0.2          0.0001         0.673190   
                                                                                      0.0005         0.687046   
                                                                                      0.0010         0.689904   
                                                          (512, 512)     0.1          0.0001         0.676980   
                                                                                      0.0005         0.693880   
                                                                                      0.0010         0.698105   
                                                                         0.2          0.0001         0.676794   
                                                                                      0.0005         0.683380   
                                                                                      0.0010         0.695061   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.652157   
                                                                                      0.0005         0.673332   
                                                                                      0.0010         0.670112   
                                                                         0.2          0.0001         0.649825   
                                                                                      0.0005         0.668642   
                                                                                      0.0010         0.665864   
                                                          (512, 512)     0.1          0.0001         0.657353   
                                                                                      0.0005         0.674935   
                                

In [47]:
best_swmhau_network_grp_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.673066,0.655999,"[0.7326219512195122, 0.5793764988009592]",0.654524,"[0.7822265625, 0.5268207588312255]",0.666257,"[0.6889334862385321, 0.6435801811401172]",None,0.857258,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.686486,0.669361,"[0.7446097783176435, 0.5941119691119691]",0.667243,"[0.7914783731439639, 0.5430083811204235]",0.679408,"[0.7029816513761468, 0.6558337773042089]",None,0.851462,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.670829,0.652348,"[0.7325053014238111, 0.5721899224806202]",0.650574,"[0.7764932562620424, 0.5246557085739671]",0.661215,"[0.6932339449541285, 0.6291955247735749]",None,0.852676,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [48]:
best_swmhau_network_grp_kfold_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001
0,15,12,3,10,1,"(512, 512)",0.2,0.0001


In [49]:
best_swmhau_network_grp_kfold_20["f1"].mean()

0.6592359035590859

In [50]:
best_swmhau_network_grp_kfold_20["precision"].mean()

0.6574471734052705

In [51]:
best_swmhau_network_grp_kfold_20["recall"].mean()

0.6689597609644514

In [52]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

array([0.73657901, 0.5818928 ])

In [53]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

array([0.7833994 , 0.53149495])

In [54]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)

array([0.69504969, 0.64286983])

# w=35

In [10]:
size = 35

## Random Projections

In [11]:
(
    swmhau_network_grp_kfold_35,
    best_swmhau_network_grp_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_kfold_best_model.csv


In [12]:
swmhau_network_grp_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1465106/2891164155.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_35.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.679466   
                                                                                      0.0005         0.702827   
                                                                                      0.0010         0.697359   
                                                                         0.2          0.0001         0.679963   
                                                                                      0.0005         0.697297   
                                                                                      0.0010         0.693632   
                                                          (512, 512)     0.1          0.0001         0.681019   
                                                                                      0.0005         0.704442   
                                                                                      0.0010         0.701087   
                                                                         0.2          0.0001         0.673936   
                                                                                      0.0005         0.699845   
                                                                                      0.0010         0.699596   
           12              3         10        1          (256, 256)     0.1          0.0001         0.685865   
                                                                                      0.0005         0.686549   
                                                                                      0.0010         0.704007   
                                                                         0.2          0.0001         0.680273   
                                                                                      0.0005         0.698975   
                                                                                      0.0010         0.699720   
                                                          (512, 512)     0.1          0.0001         0.679217   
                                                                                      0.0005         0.687232   
                                                                                      0.0010         0.706865   
                                                                         0.2          0.0001         0.683194   
                                                                                      0.0005         0.692637   
                                                                                      0.0010         0.700528   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.654539   
                                                                                      0.0005         0.679077   
                                                                                      0.0010         0.675188   
                                                                         0.2          0.0001         0.659153   
                                                                                      0.0005         0.676619   
                                                                                      0.0010         0.670943   
                                                          (512, 512)     0.1          0.0001         0.656143   
                                                                                      0.0005         0.682961   
                                

In [13]:
best_swmhau_network_grp_kfold_35

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.695993,0.672673,"[0.7600411946446961, 0.5853038393084159]",0.670212,"[0.7805983680870353, 0.5598249027237354]",0.676876,"[0.7405389908256881, 0.6132125732551944]",None,0.885564,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.687978,0.668163,"[0.7492510485320552, 0.587074494326591]",0.665564,"[0.784504391468005, 0.5466237942122186]",0.675510,"[0.7170298165137615, 0.633990410229089]",None,0.886238,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.673625,0.654508,"[0.7357778783763393, 0.5732390933463318]",0.652491,"[0.776680471487735, 0.5283018867924528]",0.662750,"[0.6989678899082569, 0.6265316995205115]",None,0.893247,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_grp_kfold_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [15]:
best_swmhau_network_grp_kfold_35["f1"].mean()

0.6651145914224048

In [16]:
best_swmhau_network_grp_kfold_35["precision"].mean()

0.662755635795197

In [17]:
best_swmhau_network_grp_kfold_35["recall"].mean()

0.6717118967087501

In [18]:
np.stack(best_swmhau_network_grp_kfold_35["f1_scores"]).mean(axis=0)

array([0.74835671, 0.58187248])

In [19]:
np.stack(best_swmhau_network_grp_kfold_35["precision_scores"]).mean(axis=0)

array([0.78059441, 0.54491686])

In [20]:
np.stack(best_swmhau_network_grp_kfold_35["recall_scores"]).mean(axis=0)

array([0.71884557, 0.62457823])

# w=80

In [10]:
size = 80

## Random Projections

In [11]:
(
    swmhau_network_grp_kfold_80,
    best_swmhau_network_grp_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_80_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_80_kfold_best_model.csv


In [12]:
swmhau_network_grp_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_3580475/3749845653.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp_kfold_80.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.690649   
                                                                                      0.0005         0.722088   
                                                                                      0.0010         0.701087   
                                                                         0.2          0.0001         0.679404   
                                                                                      0.0005         0.711712   
                                                                                      0.0010         0.705623   
                                                          (512, 512)     0.1          0.0001         0.681516   
                                                                                      0.0005         0.713948   
                                                                                      0.0010         0.708170   
                                                                         0.2          0.0001         0.696925   
                                                                                      0.0005         0.714445   
                                                                                      0.0010         0.689593   
           12              3         10        1          (256, 256)     0.1          0.0001         0.694812   
                                                                                      0.0005         0.694626   
                                                                                      0.0010         0.716682   
                                                                         0.2          0.0001         0.692016   
                                                                                      0.0005         0.702578   
                                                                                      0.0010         0.719105   
                                                          (512, 512)     0.1          0.0001         0.700466   
                                                                                      0.0005         0.708792   
                                                                                      0.0010         0.721528   
                                                                         0.2          0.0001         0.684498   
                                                                                      0.0005         0.709599   
                                                                                      0.0010         0.705809   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.665722   
                                                                                      0.0005         0.692417   
                                                                                      0.0010         0.677640   
                                                                         0.2          0.0001         0.657991   
                                                                                      0.0005         0.689324   
                                                                                      0.0010         0.680300   
                                                          (512, 512)     0.1          0.0001         0.657778   
                                                                                      0.0005         0.688379   
                                

In [13]:
best_swmhau_network_grp_kfold_80

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.680522,0.661840,"[0.7413220645940235, 0.5823586744639376]",0.659630,"[0.7826641172721479, 0.5365963179164795]",0.670391,"[0.7041284403669725, 0.6366542354821524]",None,0.901469,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.692637,0.674935,"[0.7507934109112891, 0.5990761001701921]",0.672424,"[0.7938638542665388, 0.5509838998211091]",0.684261,"[0.7121559633027523, 0.6563665423548215]",None,0.891225,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.711277,0.682936,"[0.7777299469077342, 0.5881414517415581]",0.682780,"[0.7785119218615341, 0.5870488322717622]",0.683094,"[0.7769495412844036, 0.5892381459776239]",None,0.900256,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_grp_kfold_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


In [15]:
best_swmhau_network_grp_kfold_80["f1"].mean()

0.6732369414647891

In [16]:
best_swmhau_network_grp_kfold_80["precision"].mean()

0.671611490568262

In [17]:
best_swmhau_network_grp_kfold_80["recall"].mean()

0.6792488114614544

In [18]:
np.stack(best_swmhau_network_grp_kfold_80["f1_scores"]).mean(axis=0)

array([0.75661514, 0.58985874])

In [19]:
np.stack(best_swmhau_network_grp_kfold_80["precision_scores"]).mean(axis=0)

array([0.7850133 , 0.55820968])

In [20]:
np.stack(best_swmhau_network_grp_kfold_80["recall_scores"]).mean(axis=0)

array([0.73107798, 0.62741964])